# Profiles del capitulo 2

Los datos estan disponibles en https://bioinformaticsalgorithms.com/data/realdatasets/Motifs/subtle_motif_dataset.txt. En mi caso los guarde en un archivo llamado "secuencias.txt".

Importar librerias que nos haran falta luego.

In [1]:
import random
import copy

Dos maneras de leer los datos, con un for o con un "list comprehension". Hacen lo mismo

In [2]:
lines = []

with open("secuencias.txt") as f:
    for line in f.readlines():
        line = line.strip()
        line = line.replace("*", "")
        lines.append(line)

In [72]:
with open("/Users/humberto/Downloads/rosalind_ba2g.txt") as f:
    lines = [ line.strip().replace("*", "") for line in  f.readlines()]


In [73]:
lines

['15 20 2000',
 'ACATACGTCGACCGCGAGGTGCATGATCATGGACGTGTCGCCAACTCGATCTCATAGAAAAGCGTAAAGCTTCTACAAGCATTGAGTCTCCCGAGGGAGTTTGTCACGTTTTCCAAATAATCATGATTCCGCCATGTTCGCACGTCACTCGATGGAGCGCAGGGTAGTGCCGGTAACAGGTACTAACGCAGGTTCAGATCCTATGGAGCAAGCCTTGTCGTCCCAGTGACGGCGGTCGCGACATTTGGAAGGGTGCTTCACTCCGCGATATTTAAAGTTACCGATGGGCCCAACTGTTGTGAACGGGACATACGTCGACCGC',
 'GAGGTGCATGATCATGGACGTGTCGCCAACTCGATCTCATAGAACCGGGGGACAAGGACAAGCGTAAAGCTTCTACAAGCATTGAGTCTCCCGAGGGAGTTTGTCACGTTTTCCAAATAATCATGATTCCGCCATGTTCGCACGTCACTCGATGGAGCGCAGGGTAGTGCCGGTAACAGGTACTAACGCAGGTTCAGATCCTATGGAGCAAGCCTTGTCGTCCCAGTGACGGCGGTCGCGACATTTGGAAGGGTGCTTCACTCCGCGATATTTAAAGTTACCGATGGGCCCAACTGTTGTGAACGGGACATACGTCGACCGC',
 'CAGACAGCACCCGTTAGATCCGGACACCTAGAGGCACTTGAGAAACCCCATCCAAAATTGGTTATACCGATCAGTTCCCAGGTTCTTATCGTTTCATTCCAGAGCCGGCCTCAGTCGGAACTTGAATCGGTCCAGTGTGCATAAGGAAGGTTTGCTTCTTCCACGTACGCGTTGCCATCCTGTCGTCAACGCACCTCGCGAATAGTGTCGGACGGCGGCTCGGCGGTGCAGATGTCTACCTACGGTGCACTACACGCAATTCGCCCAGACGCAACGAGAATACGCATGCGAGGGGGATATCTCAATGGGTACGTTTAGTCTT',
 'CA

In [5]:
len(lines[0])

600

In [6]:
random.randint(0,len(lines[0])-15)

32

Una funcion para generar un k-mero random de una secuencia.

In [7]:
def randomkmer(line, k):
    start = random.randint(0, len(line)-k-1)
    return line[start:start+k]

In [8]:
randomkmer(lines[0], 15)

'GTCCAACAATGTTAG'

In [9]:
Motif = [randomkmer(line, 15) for line in lines]

In [10]:
Motif

['GGAGGGCTGAAATAC',
 'AAAAATAGCAGGGTG',
 'GTTGGACTCTCCAAC',
 'TACGGCGGTTTTCGT',
 'TTCACGGGAGAAAGA',
 'ACTTAAGCGTTCGAA',
 'CAAGAGAGGTGTATC',
 'ATGCCCTGGAAGGGA',
 'CGGGTTTTAAGCGGG',
 'CGCAGGACCAAAGCC']

In [11]:
random.randint(0,20)

7

In [12]:
def MakeCounts(motif):
    k = len(motif[0])
    counts = [[1 for i in range(k)] for j in range(4) ]
    for i in range(k):
        for j in range(len(motif)):
            letra = motif[j][i]
            index = "ACGT".index(letra)
            counts[index][i] += 1
    return counts

In [13]:
# Arreglamos un bug aqui. El denominador es 4+filas, no el doble de filas
def MakeProfile(motif):
    Counts = MakeCounts(motif)
    Profile = copy.deepcopy(Counts)
    filas = len(motif)
    for i in range(len(Counts)):
        for j in range(len(Counts[0])):
            Profile[i][j] = Counts[i][j]/(4+filas)
    return Profile

In [115]:
def MakeKmer(Motifs, Text, k):
    counts = MakeCounts(Motifs)
    probs = []
    for i in range(len(Text) - k + 1):
        kmer = Text[i:i+k]
        prob = 1
        for j in range(k):
            row = "ACGT".index(kmer[j])
            prob *= counts[row][j]
        probs.append(prob)
    element = random.choices(range(len(probs)), weights = probs)[0]
    
    return Text[element:element+k]

In [15]:
# Esta funcion aun esta rota
def Score(Motifs):
    counts = MakeCounts(Motifs)
    score = 0
    k = len(Motifs[0])
    for i in range(k):
        lista = []
        for j in range(4):
            cell = counts[j][i] - 1
            score += cell
            lista.append(cell)

        top = max(lista)
        score -= top

    return score

In [113]:
def GibbsSampler(DNA, k, t, N):
    # pick a random start
    Motif = [randomkmer(line, k) for line in DNA]
    BestMotif = Motif[:]
    for j in range(N):
        # seleccionar una fila del motif
        i = random.randint(0, t-1)
        #Motif[i] = randomkmer(DNA[i], k) # esto es como del 2.7 (random search)
        newmotif = Motif[:] # hacer copia de Motif
        Text = newmotif.pop(i) # eliminar la fila i
        
        Profile = MakeProfile(newmotif)
        newkmer = MakeKmer(newmotif, Text, k)
        Motif[i] = newkmer
        # Computar score nuevo motif
        # Si es mejor cambiar BestMotif por Motif
        if Score(Motif) < Score(BestMotif):
            BestMotif = Motif[:]
 
    return BestMotif

# Pruebas 

Algunas pruebas para ver si entiendo python y como hacer algunas cosas.

In [17]:
Motif

['GGAGGGCTGAAATAC',
 'AAAAATAGCAGGGTG',
 'GTTGGACTCTCCAAC',
 'TACGGCGGTTTTCGT',
 'TTCACGGGAGAAAGA',
 'ACTTAAGCGTTCGAA',
 'CAAGAGAGGTGTATC',
 'ATGCCCTGGAAGGGA',
 'CGGGTTTTAAGCGGG',
 'CGCAGGACCAAAGCC']

In [18]:
i = 3

Motif[i] = randomkmer(lines[i], 15)

In [19]:
Motif

['GGAGGGCTGAAATAC',
 'AAAAATAGCAGGGTG',
 'GTTGGACTCTCCAAC',
 'GGGGCCCTCCAGCTT',
 'TTCACGGGAGAAAGA',
 'ACTTAAGCGTTCGAA',
 'CAAGAGAGGTGTATC',
 'ATGCCCTGGAAGGGA',
 'CGGGTTTTAAGCGGG',
 'CGCAGGACCAAAGCC']

In [20]:
Motif.pop(3)

'GGGGCCCTCCAGCTT'

In [21]:
Motif

['GGAGGGCTGAAATAC',
 'AAAAATAGCAGGGTG',
 'GTTGGACTCTCCAAC',
 'TTCACGGGAGAAAGA',
 'ACTTAAGCGTTCGAA',
 'CAAGAGAGGTGTATC',
 'ATGCCCTGGAAGGGA',
 'CGGGTTTTAAGCGGG',
 'CGCAGGACCAAAGCC']

In [22]:
"ACGT".index("A")


0

In [23]:
"ACGT".index("C")

1

In [24]:
counts = [[1] * 15] * 4

In [25]:
otros_counts = [[1 for i in range(15)] for j in range(4) ]

In [26]:
MakeCounts(Motif)

[[4, 3, 4, 4, 4, 3, 4, 1, 3, 6, 5, 4, 4, 4, 4],
 [4, 2, 3, 2, 3, 2, 3, 3, 4, 1, 2, 4, 1, 2, 5],
 [3, 4, 3, 5, 4, 5, 3, 5, 5, 2, 4, 3, 6, 4, 3],
 [2, 4, 3, 2, 2, 3, 3, 4, 1, 4, 2, 2, 2, 3, 1]]

In [27]:
counts

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [28]:
counts[0][3] += 1

In [29]:
counts

[[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [30]:
otros_counts

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [31]:
otros_counts[0][3] += 1

In [32]:
otros_counts

[[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [33]:
otros_otros_counts = copy.deepcopy(otros_counts)

In [34]:
otros_counts[0][0] = 23

In [35]:
otros_counts

[[23, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [36]:
otros_otros_counts

[[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [37]:
MakeProfile(Motif)

[[0.3076923076923077,
  0.23076923076923078,
  0.3076923076923077,
  0.3076923076923077,
  0.3076923076923077,
  0.23076923076923078,
  0.3076923076923077,
  0.07692307692307693,
  0.23076923076923078,
  0.46153846153846156,
  0.38461538461538464,
  0.3076923076923077,
  0.3076923076923077,
  0.3076923076923077,
  0.3076923076923077],
 [0.3076923076923077,
  0.15384615384615385,
  0.23076923076923078,
  0.15384615384615385,
  0.23076923076923078,
  0.15384615384615385,
  0.23076923076923078,
  0.23076923076923078,
  0.3076923076923077,
  0.07692307692307693,
  0.15384615384615385,
  0.3076923076923077,
  0.07692307692307693,
  0.15384615384615385,
  0.38461538461538464],
 [0.23076923076923078,
  0.3076923076923077,
  0.23076923076923078,
  0.38461538461538464,
  0.3076923076923077,
  0.38461538461538464,
  0.23076923076923078,
  0.38461538461538464,
  0.38461538461538464,
  0.15384615384615385,
  0.3076923076923077,
  0.23076923076923078,
  0.46153846153846156,
  0.3076923076923077,
  

In [38]:
"ACGT*CCTTA".replace("*", "")

'ACGTCCTTA'

In [39]:
.16+.11+.27+.16

0.7000000000000001

In [40]:
MakeCounts(Motif)

[[4, 3, 4, 4, 4, 3, 4, 1, 3, 6, 5, 4, 4, 4, 4],
 [4, 2, 3, 2, 3, 2, 3, 3, 4, 1, 2, 4, 1, 2, 5],
 [3, 4, 3, 5, 4, 5, 3, 5, 5, 2, 4, 3, 6, 4, 3],
 [2, 4, 3, 2, 2, 3, 3, 4, 1, 4, 2, 2, 2, 3, 1]]

In [41]:
len(Motif)

9

In [42]:
Motif

['GGAGGGCTGAAATAC',
 'AAAAATAGCAGGGTG',
 'GTTGGACTCTCCAAC',
 'TTCACGGGAGAAAGA',
 'ACTTAAGCGTTCGAA',
 'CAAGAGAGGTGTATC',
 'ATGCCCTGGAAGGGA',
 'CGGGTTTTAAGCGGG',
 'CGCAGGACCAAAGCC']

In [43]:
.46+0.07+.07+.38


0.9800000000000001

In [44]:
Profile = MakeProfile(Motif)

In [45]:
Profile

[[0.3076923076923077,
  0.23076923076923078,
  0.3076923076923077,
  0.3076923076923077,
  0.3076923076923077,
  0.23076923076923078,
  0.3076923076923077,
  0.07692307692307693,
  0.23076923076923078,
  0.46153846153846156,
  0.38461538461538464,
  0.3076923076923077,
  0.3076923076923077,
  0.3076923076923077,
  0.3076923076923077],
 [0.3076923076923077,
  0.15384615384615385,
  0.23076923076923078,
  0.15384615384615385,
  0.23076923076923078,
  0.15384615384615385,
  0.23076923076923078,
  0.23076923076923078,
  0.3076923076923077,
  0.07692307692307693,
  0.15384615384615385,
  0.3076923076923077,
  0.07692307692307693,
  0.15384615384615385,
  0.38461538461538464],
 [0.23076923076923078,
  0.3076923076923077,
  0.23076923076923078,
  0.38461538461538464,
  0.3076923076923077,
  0.38461538461538464,
  0.23076923076923078,
  0.38461538461538464,
  0.38461538461538464,
  0.15384615384615385,
  0.3076923076923077,
  0.23076923076923078,
  0.46153846153846156,
  0.3076923076923077,
  

In [46]:
r = random.random()

In [47]:
r

0.14318199374021334

In [48]:
Profile[0][0]

0.3076923076923077

In [49]:
Profile[1][0]

0.3076923076923077

In [50]:
Profile[2][0]

0.23076923076923078

In [51]:
Profile[3][0]

0.15384615384615385

In [52]:
""+ "A"

'A'

In [53]:
kmer = ""

In [54]:
kmer += "A"

In [55]:
kmer

'A'

In [56]:
MakeKmer(Profile, 15)

'CATCAGAGGATGAGC'

In [57]:
Score(Motif)

80

In [58]:
lista = [6,1,1,5]

In [59]:
max(lista)

6

In [70]:
GibbsSampler(lines, 15, 10, 2000)

['GTGTCGTAAACTGAC',
 'GGGTAGACTATTCCC',
 'GGGAAGTGGTGGTTA',
 'GGAGATGTGGATACC',
 'CGAATTACGAATAGC',
 'TGATATACGTCGAAC',
 'CGGTCGTCGCCTACC',
 'GGGGAGCCCACCAGC',
 'TCAATGACGATTCTT',
 'TCTCATCCGTAGATT']

In [109]:
# Test on rosalind data
shortdna = ["CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA",
"GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG",
"TAGTACCGAGACCGAAAGAAGTATACAGGCGT",
"TAGATCAAGTTTCAGGTGCACGTCGGTGAACC",
"AATCCACCAGCTCCACGTGCAATGTTGGCCTA"]

In [116]:
for sequence in GibbsSampler(shortdna, 8, 5,100):
    print(sequence)

GGGGTGTT
AGTGCCAA
TACCGAGA
ATCAAGTT
CCAGCTCC


In [74]:
lines[0]

'15 20 2000'

In [75]:
lines.pop(0)

'15 20 2000'

In [76]:
for sequence in GibbsSampler(lines, 15, 20,2000):
    print(sequence)

CGCCATGTTCGCACG
AGGGTAGTGCCGGTA
GCACTTGAGAAACCC
TTATCGGAGCAAGAC
ACGGAAGAGAAAACT
GGTCCTTGTAATGCC
AGGAGTATTAAGTGC
TTGGCCGATTGTCTA
CACGCTGCAAAGGTG
GTGCTCTTGTAGACT
CTAGCAGGCGAGCCC
CTCTCGGCCCCGGAC
GTGGTTATTCTTGCC
CCGTTACGTATGGCC
CAAGGTTTTTAAGCC
CCCCGTTCTATAGAT
CTAAGTCGTTCCTTG
CTGGTTGAGGAAATG
ATGGCCGATTGAGTA
CGCGTTAAACAGACA


In [78]:
Motif

['GGAGGGCTGAAATAC',
 'AAAAATAGCAGGGTG',
 'GTTGGACTCTCCAAC',
 'TTCACGGGAGAAAGA',
 'ACTTAAGCGTTCGAA',
 'CAAGAGAGGTGTATC',
 'ATGCCCTGGAAGGGA',
 'CGGGTTTTAAGCGGG',
 'CGCAGGACCAAAGCC']

In [79]:
MakeKmer(Motif, lines[0], 15)

[19660800,
 167961600,
 6635520,
 4976640,
 19906560,
 19906560,
 139968000,
 37324800,
 66355200,
 311040,
 414720000,
 4423680,
 69120000,
 186624000,
 106168320,
 324000,
 106168320,
 55987200,
 2488320,
 265420800,
 17915904,
 829440,
 132710400,
 12441600,
 20995200,
 3317760,
 24576000,
 31104000,
 22118400,
 5529600,
 70778880,
 3888000,
 10368000,
 49766400,
 11197440,
 663552,
 24883200,
 165888,
 16588800,
 19906560,
 3317760,
 442368,
 53084160,
 3317760,
 637009920,
 19906560,
 21233664,
 19906560,
 124416000,
 74649600,
 3732480,
 53084160,
 7962624,
 39321600,
 53084160,
 884736000,
 62208000,
 14929920,
 93312000,
 331776,
 1382400,
 44236800,
 10616832,
 5898240,
 141557760,
 74649600,
 14929920,
 1658880,
 13436928,
 1492992,
 2457600,
 59719680,
 15925248,
 24883200,
 1966080,
 93312000,
 12441600,
 2764800,
 460800,
 21233664,
 24883200,
 15925248,
 26542080,
 50388480,
 89579520,
 7464960,
 1244160,
 25920000,
 18662400,
 6912000,
 41472000,
 49152000,
 12441600,
 2

In [81]:
DNA = ["ttACCTtaac",
"gATGTctgtc",
"ccgGCGTtag",
"cactaACGAg",
"cgtcagAGGT"]

In [82]:
DNA

['ttACCTtaac', 'gATGTctgtc', 'ccgGCGTtag', 'cactaACGAg', 'cgtcagAGGT']

In [84]:
DNA = [x.upper() for x in DNA]

In [85]:
DNA

['TTACCTTAAC', 'GATGTCTGTC', 'CCGGCGTTAG', 'CACTAACGAG', 'CGTCAGAGGT']

In [86]:
Motifs = ["taac",
"GTct",
"acta",
"AGGT"]

In [87]:
Motifs = [x.upper() for x in Motifs]

In [88]:
Motifs

['TAAC', 'GTCT', 'ACTA', 'AGGT']

In [95]:
x = MakeKmer(Motifs, DNA[2], 4)

In [97]:
[[i]*x[i] for i in range(len(x))]

[[0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2, 2, 2, 2],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 [6, 6, 6, 6, 6, 6, 6, 6]]

In [106]:
random.choices(range(len(x)), weights = x)

[2]